In [20]:
import pandas as pd
import numpy as np
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler, Subset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torch.optim as optim
from torchvision import models
from torchvision.models import vit_b_16, ViT_B_16_Weights
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from torchmetrics.classification import MulticlassF1Score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
base_dir = "Images/FloodNet Challenge - Track 1"

In [22]:
# Define data augmentations and preprocessing
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet stats
])


In [23]:
# Load the labeled training data
train_dataset = datasets.ImageFolder(f'{base_dir}/Train/Labeled', transform=train_transforms)
# Get the class names and their corresponding indices
class_to_idx = train_data.class_to_idx
print(f"Class to Index Mapping: {class_to_idx}")

# Count the samples in each class
class_counts = {class_name: 0 for class_name in class_to_idx.keys()}
for _, label in train_data.samples:
    for class_name, class_idx in class_to_idx.items():
        if label == class_idx:
            class_counts[class_name] += 1

print("Class Distribution:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")

Class to Index Mapping: {'Flooded': 0, 'Non-Flooded': 1}
Class Distribution:
Flooded: 51
Non-Flooded: 347


In [24]:
# Large class imbalance so need to use WeightedRandomSampler to ensure balanced mini-batches
# during training.

# Extract class labels from the dataset
targets = train_dataset.targets

# Calculate class weights, which are the inverse of class frequencies. Classes with fewer samples will get assigned a higher 
# weight (ensuring that the minority class receives a higher weight, making it more likely to be sampled during training).
class_counts = np.bincount(targets)  # Count the number of samples per class
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

sample_weights = torch.tensor([class_weights[label] for label in targets], dtype=torch.float)

# Create the WeightedRandomSampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoaders with the sampler for training
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4)

print(f"Number of training samples: {len(train_loader.dataset)}")

Number of training samples: 398


/tmp/ipykernel_5443/3797760210.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)


In [ ]:
# Import the ViT model for transfer learning
model = models.vit_b_16(weights=ViT_B_16_Weights.DEFAULT )
# Freeze the parameters in the base model so only the new layers are being updated
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer
num_classes = len(class_counts)

model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)

# Unfreeze the final layer to allow it to learn during training
for param in model.heads.head.parameters():
    param.requires_grad = True

model = model.to(device)

# Initialize the loss function with the class weights to penalize errors made on the minority class more heavily.
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Set up the optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

# Set the number of epochs for training
num_epochs = 10

# Metrics tracker
f1_metric = MulticlassF1Score(num_classes=num_classes, average='weighted').to(device)

# Store the model with the best performance on training for use on the validation set
best_train_loss = float('inf')
best_model_path = "best_model.pth"  # Path to save the best model based on training loss

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    f1_metric.reset()

    for images, labels in train_loader:  # Iterate over DataLoader
        images, labels = images.to(device), labels.to(device)  # Move to device

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)  # Calculate loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        f1_metric.update(preds, labels)

    # Calculate average training loss for this epoch
    avg_train_loss = running_loss / len(train_loader)
    f1_score = f1_metric.compute()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_train_loss:.4f}, F1 Score: {f1_score:.4f}")

    # Save the model if it has the lowest training loss
    if avg_train_loss < best_train_loss:
        best_train_loss = avg_train_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with training loss: {best_train_loss:.4f}")

Epoch [1/10], Loss: 0.3853, F1 Score: 0.4793
New best model saved with training loss: 0.3853
Epoch [2/10], Loss: 0.2069, F1 Score: 0.4984
New best model saved with training loss: 0.2069
Epoch [3/10], Loss: 0.1841, F1 Score: 0.7557
New best model saved with training loss: 0.1841
Epoch [4/10], Loss: 0.1425, F1 Score: 0.7746
New best model saved with training loss: 0.1425
Epoch [5/10], Loss: 0.1242, F1 Score: 0.8649
New best model saved with training loss: 0.1242
Epoch [6/10], Loss: 0.1134, F1 Score: 0.8748
New best model saved with training loss: 0.1134
Epoch [7/10], Loss: 0.0898, F1 Score: 0.9068
New best model saved with training loss: 0.0898
Epoch [8/10], Loss: 0.0981, F1 Score: 0.9295
Epoch [9/10], Loss: 0.0644, F1 Score: 0.9495
New best model saved with training loss: 0.0644
